In [10]:
%matplotlib inline
import numpy as np
import pylab as pl
import scipy.special as special
from scipy.integrate import quad
from scipy.integrate import nquad
import math as math

# Set plot parameters to make beautiful plots
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20  
pl.rcParams['font.sans-serif'] = 'serif'
#pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'medium'

pl.rcParams['xtick.major.size'] = 8     
pl.rcParams['xtick.minor.size'] = 4     
pl.rcParams['xtick.major.pad']  = 8     
pl.rcParams['xtick.minor.pad']  = 8     
pl.rcParams['xtick.color']      = 'k'     
pl.rcParams['xtick.labelsize']  = 'medium'
pl.rcParams['xtick.direction']  = 'in'    

pl.rcParams['ytick.major.size'] = 8     
pl.rcParams['ytick.minor.size'] = 4     
pl.rcParams['ytick.major.pad']  = 8     
pl.rcParams['ytick.minor.pad']  = 8     
pl.rcParams['ytick.color']      = 'k'     
pl.rcParams['ytick.labelsize']  = 'medium'
pl.rcParams['ytick.direction']  = 'in'    

In [17]:
#want to plot the integrand in Eq. 60 of the paper
#with filled in values of I(...), delta(yn), 

#constants in natural units
#m = 1.
#c = 1.
theta_e = 10.
#nu_c = 1
#B = 10.
#e = 1


#constants in Gaussian cgs units (values from harmony)
m = 9.1093826e-28
c = 2.99792458e10
e = 4.80320680e-10
B = 30.0
#n_e = 100000.




#variables to be set (frequency, observer angle, width of delta function, electron number density, 
# sum index n)


theta = np.pi/3.
epsilon = 0.01
n_e = 1
nu_c = e * B / (2. * np.pi * m * c)
nu = 1. * nu_c
nu_s = (2./9.) * nu_c * theta_e**2 * np.sin(theta)
n_e_nt = 1.
#p_PL = 2.2
p_PL = 3.
gamma_50_min = 1.
gamma_50_max = 1000.
#gamma_50_max = 100.
C = 1.
n_dn = (nu / nu_c) * abs(np.sin(theta))
Index_cutoff = 0.001
gamma_cutoff = 1000.




In [18]:
def K_s(n, gamma):
    beta = np.sqrt(1. - 1./gamma**2.)
    cos_xi = (gamma * nu - n * nu_c) / (gamma * nu * beta * np.cos(theta))
    M = (np.cos(theta) - beta * cos_xi)/np.sin(theta)
    N = beta * np.sqrt(1. - cos_xi**2)
    z = (nu * gamma * beta * np.sin(theta) * np.sqrt(1. - cos_xi**2))/nu_c
    K_xx = M**2 * (special.jv(n, z))**2
    K_yy = N**2 * (special.jvp(n, z, 1))**2
    ans = K_xx + K_yy
    return ans

def gamma_up_limit(n):
    gamma_up_nume = (n * nu_c /nu) + np.absolute(np.cos(theta)) * np.sqrt((n * nu_c /nu)**2 - (np.sin(theta))**2)
    gamma_deno = (np.sin(theta))**2
    gamma_up = gamma_up_nume / gamma_deno
    return gamma_up

def gamma_dn_limit(n):
    gamma_dn_nume = (n * nu_c /nu) - np.absolute(np.cos(theta)) * np.sqrt((n * nu_c /nu)**2 - (np.sin(theta))**2)
    gamma_deno = (np.sin(theta))**2
    gamma_dn = gamma_dn_nume / gamma_deno
    return gamma_dn

def power_law_tail_with_cutoff_no_norm(gamma):
    
    f_prefactor = n_e_nt * (p_PL - 1.) / (gamma_50_min**(1. - p_PL) - gamma_50_max**(1. - p_PL))
    f_need_to_be_normalized = gamma**(- p_PL) * np.exp(- gamma/gamma_cutoff)
    f = f_prefactor * f_need_to_be_normalized
    
    return f

def power_law_tail_normalized():
    ans = quad(power_law_tail_with_cutoff_no_norm, 1, np.inf,  epsabs = 0, epsrel = 1e-8)[0]
    return ans

power_law_tail_normalized_value = power_law_tail_normalized()
#print 1./power_law_tail_normalized_value

def power_law_with_cutoff_with_norm(gamma):
    beta = np.sqrt(1. - 1./gamma**2.)
    Prefactor_A = 1./power_law_tail_normalized_value
    Prefactor_47 = 1. / (m**3 * c**3 * gamma**2 * beta)
    power_law_dist = Prefactor_A * Prefactor_47 * power_law_tail_with_cutoff_no_norm(gamma) / (4. * np.pi)
    return power_law_dist


def I_in_eq62(gamma, n):
    
    beta = np.sqrt(1. - 1./gamma**2.)
    cos_xi = (gamma * nu - n * nu_c) / (gamma * nu * beta * np.cos(theta))
    
    prefactor_Jnu = 2. * np.pi * e**2 * nu**2 / c
    d_3_p = m**3 * c**3 * gamma**2 * beta * 2 * np.pi
    
    ans = prefactor_Jnu * power_law_with_cutoff_with_norm(gamma) * d_3_p * K_s(n, gamma)
    
    return ans

def jnu_integrand_in_eq62(gamma, n):
    
    beta = np.sqrt(1. - 1./gamma**2.)
    cos_xi = (gamma * nu - n * nu_c) / (gamma * nu * beta * np.cos(theta))
    
    prefactor_62 = 1. / (nu * beta * np.absolute(np.cos(theta)))
    
    f = prefactor_62 * I_in_eq62(gamma, n)
    
    return f

In [21]:
def n_peak():
    
    n_overpeak = 91. * 21**(np.log10(nu/nu_c))

    x_value = []
    y_value = []

    x_value_max = 0
    y_value_max = 0
    
    x_variable = 0

    if n_overpeak < 1000:
        step_div = n_overpeak
    else:
        step_div = 1000
        
    step = int((int(n_overpeak))/step_div)


    for i in xrange(int(n_dn + 1.), int(n_dn + 1. + n_overpeak), step):
        x_value = x_value + [i]
        y_value = y_value + [jnu_n_integral_62_one(i)]
    
    for j in xrange(0, int(step_div), 1):
        if y_value[j] > y_value_max:
            x_value_max = x_value[j]
            y_value_max = y_value[j]
            x_variable = j
    
    y_min = (1./1000) * y_value_max
    
#    print x_value_max
    
    
    for k in xrange(x_variable, int(step_div), 1):
        if y_value[k] < y_min:
            x_mini = x_value[k]
            return x_mini


        
def jnu_n_integral_62_one(n):
    
#    def jnu_integrand_in_eq62_gamma(gamma):
#        jnu_integrand_62 = jnu_integrand_in_eq62(gamma, n)
#        return jnu_integrand_62
    
    
    ans_once = 0
    
    gamma_low = gamma_dn_limit(n)
    gamma_high= gamma_up_limit(n)
    
    step = 1
    
    
    for i in xrange(int(gamma_low * 1e3), int(gamma_high * 1e3), step):
        ans_once = ans_once + jnu_integrand_in_eq62(float(i/1e3), n)
    
    
    return ans_once

#def jnu_nonthermal_power_law():
#    ans = 0
#    for i in range(int(n_dn + 1.), int(n_dn + 100000.)):
#        ans = ans + jnu_n_integral_62_one(i)
#    return ans

#n_peak = 3000000000

def jnu_62_integral_from_n_max_to_n_limit(n_max, n_dn):
    
    if(n_max < n_dn):
        n_max = n_dn
    
    ans = quad(jnu_n_integral_62_one, n_max, C * n_peak(), epsabs = 0, epsrel = 1e-8)[0]
    
    return ans
    

def jnu_62_sum_to_n_max_integral_to_n_limit(n_max_sum):
    ans = 0
    for i in range(int(n_dn + 1.), int(1 + n_dn + n_max_sum)):
        ans = ans + jnu_n_integral_62_one(i)
#        print ans
        
    total = ans + jnu_62_integral_from_n_max_to_n_limit(n_max_sum, n_dn)
    return total




n_peak()

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:62: RuntimeWarning: invalid value encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:64: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:50: RuntimeWarning: invalid value encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:42: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:55: RuntimeWarning: invalid value encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: RuntimeWarning: invalid value encountered in sqrt
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: RuntimeWarn

KeyboardInterrupt: 

In [14]:
gamma_dn_limit(n_dn + 1.), gamma_up_limit(n_dn + 1.)

(1.0800426901451148, 1.2560250532800556)

In [20]:
print jnu_62_sum_to_n_max_integral_to_n_limit(30)

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:62: RuntimeWarning: invalid value encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:64: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:50: RuntimeWarning: invalid value encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:42: RuntimeWarning: divide by zero encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:55: RuntimeWarning: invalid value encountered in double_scalars
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: RuntimeWarning: invalid value encountered in sqrt
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: RuntimeWarn

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'